<a href="https://colab.research.google.com/github/Wiktor-Zagaj/Raport10/blob/main/Untitled10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from csv import reader
!wget https://raw.githubusercontent.com/WeronikaSta/Raport-10/refs/heads/main/googleplaystore.csv -O google_wget.csv
!wget https://raw.githubusercontent.com/WeronikaSta/Raport-10/refs/heads/main/AppleStore.csv-O apple_wget.csv


def read_data(filename):
    with open(filename) as f:
        return list(reader(f))

def clean_bad_rows(dataset):
    correct_length = len(dataset[0])
    cleaned=[]
    for row in dataset:
      if len(row) == correct_length:
        cleaned.append(row)
    return cleaned

def remove_duplicates(dataset, name_index, reviews_index):
    reviews_max = {}
    for row in dataset[1:]:
        name = row[name_index]
        n_reviews = float(row[reviews_index])
        if name not in reviews_max or n_reviews > reviews_max[name]:
            reviews_max[name] = n_reviews

    cleaned = []
    already_added = []
    for row in dataset[1:]:
        name = row[name_index]
        n_reviews = float(row[reviews_index])
        if reviews_max[name] == n_reviews and name not in already_added:
            cleaned.append(row)
            already_added.append(name)
    return  cleaned

def is_english(string):
    non_ascii = 0
    for c in string:
        if ord(c) > 127:
            non_ascii += 1
        if non_ascii > 3:
            return False
    return True

def filter_english_apps(dataset, name_index):
    filtered=[]
    for row in dataset:
      if is_english(row[name_index]):
        filtered.append(row)
    return filtered

def filter_free_apps(dataset, price_index, free_signature):
    free_apps=[]
    for row in dataset:
      if row[price_index] == free_signature:
        free_apps.append(row)
    return free_apps

def prepare_google_data():
    data = read_data('google_wget.csv')
    data = clean_bad_rows(data)
    data = remove_duplicates(data, name_index=0, reviews_index=3)
    data = filter_english_apps(data, name_index=0)
    data = filter_free_apps(data, price_index=7, free_signature='0')
    return data

def prepare_apple_data():
    data = read_data('apple_wget.csv')
    data = clean_bad_rows(data)
    data = filter_english_apps(data, name_index=1)
    data = filter_free_apps(data, price_index=4, free_signature='0.0')
    return data

def genre_avg_ratings(dataset, genre_index, ratings_index):
    genre_data = {}

    for row in dataset:
        genre = row[genre_index]
        try:
          rating = float(row[ratings_index])
        except:
          rating= row[ratings_index]
          rating = rating.replace(',', '')
          rating = rating.replace('+', '')
          rating = float(rating)

        if genre in genre_data:
            genre_data[genre]['total_ratings'] += rating
            genre_data[genre]['count'] += 1
        else:
            genre_data[genre] = {'total_ratings': rating, 'count': 1}

    avg_ratings = {}
    for genre in genre_data:
        total = genre_data[genre]['total_ratings']
        count = genre_data[genre]['count']
        avg_ratings[genre] = round(total / count)

    return sorted(avg_ratings.items(), key=lambda x: x[1], reverse=True)

def verify_step12_2():
 return unich_step12_2() == genre_avg_ratings(prepare_apple_data(),-5 ,5)

def verify_step13_2():
  return unich_step13_2() == genre_avg_ratings(prepare_google_data(),1,5)

print(verify_step12_2())
print(verify_step13_2())